In [ ]:
import pandas as pd
import panel as pn
import plotly.express as px
import os

pn.extension('plotly')

# --------------------------------------------
# Load and Clean Data
# --------------------------------------------
file_path = "combined_resources_file.csv"

if not os.path.exists(file_path):
    raise FileNotFoundError("❌ CSV file not found. Make sure 'combined_resources_file.csv' is in the working directory.")

df = pd.read_csv(file_path)
df.columns = [col.strip() for col in df.columns]  # Strip whitespace from column names

# Define identifying columns
exclude_cols = ["Location Name", "Formal Name", "Max Capacity"]
dummy_columns = [col for col in df.columns if col not in exclude_cols]

# Clean dummy columns: replace blanks/NaNs with 0, convert to numeric
df[dummy_columns] = df[dummy_columns].replace(r'^\s*$', 0, regex=True).fillna(0)
df[dummy_columns] = df[dummy_columns].apply(pd.to_numeric, errors='coerce').fillna(0)

# Keep only dummy columns with non-zero total counts
dummy_columns = [col for col in dummy_columns if df[col].sum() > 0]

# --------------------------------------------
# Define Widgets
# --------------------------------------------
feature_search = pn.widgets.AutocompleteInput(
    name='Room Feature',
    options=sorted(dummy_columns),
    placeholder='Type exact feature name (case-sensitive)',
    case_sensitive=True,
    restrict=True,
    width=400
)

# --------------------------------------------
# Define Callback Function
# --------------------------------------------
@pn.depends(feature_search)
def update_bar(selected_feature):
    if not selected_feature or selected_feature not in df.columns:
        return pn.pane.Markdown("⚠️ Please select a valid room feature above. KEYWORDS: FURNITURE-TECHNOLOGY-LAB ")

    # Group by Location Name and sum values
    sum_df = df.groupby("Location Name")[selected_feature].sum().reset_index()
    sum_df.columns = ['Location Name', 'Total']
    sum_df = sum_df[sum_df['Total'] > 0]

    # Campus group classification
    def assign_group(name):
        name = str(name).upper()
        if 'MH' in name or 'MEH' in name or 'VIC' in name:
            return 'DF'
        elif 'BX' in name:
            return 'BX'
        elif 'MT' in name:
            return 'MT'
        else:
            return 'Other'

    sum_df['CampusGroup'] = sum_df['Location Name'].apply(assign_group)

    fig = px.bar(
        sum_df,
        x='Location Name',
        y='Total',
        title=f"🔎 Room Feature Usage: '{selected_feature}' by Location",
        labels={'Total': 'Total Instances', 'Location Name': 'Location'},
        color='CampusGroup',
        color_discrete_map={'DF': 'dodgerblue', 'BX': 'seagreen', 'MT': 'goldenrod', 'Other': 'lightgray'}
    )
    fig.update_layout(xaxis_tickangle=45, height=600)

    return pn.pane.Plotly(fig)

# --------------------------------------------
# Dashboard Layout
# --------------------------------------------
dashboard = pn.Column(
    pn.pane.Markdown("# 🏢 Room Feature Explorer"),
    pn.pane.Markdown("Search and visualize room features by campus location."),
    feature_search,
    update_bar,
    sizing_mode='stretch_width'
)

dashboard.servable()
